In [1]:
import rclpy
import matplotlib.pyplot as plt
import numpy as np
from subscriber import Subscriber, get_observation
# from publisher import Publisher
from sensor_msgs.msg import Image
from std_msgs.msg import String, Float32MultiArray
from time import sleep

In [2]:
# rclpy.init(args=None)
# node = Subscriber()
# msg = wait_for_message(node, Image, '/Camera_wrist_rgb')
# wrist_image_array = np.array(msg.data).reshape((512, 512, 3))
# plt.imshow(wrist_image_array)
# msg = wait_for_message(node, String, '/language_topic')
# print(msg.data)
# rclpy.shutdown()

In [2]:
import os
os.environ['TOKENIZERS_PARALLELISM'] = 'false'

import cv2
import jax
import tensorflow_datasets as tfds
import tqdm
import mediapy
import numpy as np

/home/irobotics/anaconda3/envs/octo/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
PATH_CHECKPOINTS = "/media/irobotics/Transcend/finetuned_checkpoints/v4_checkpoints/"
PATH_DATASET_ROSBAG = "/media/irobotics/Transcend/isaacsim_data/v4_test/"
PATH_DATASET_TFDS = '/media/irobotics/Transcend/tensorflow_datasets/v4_test/example_dataset/1.0.0/'
PATH_INFERENCE_RESULTS = "/media/irobotics/Transcend/inference_result/"

In [4]:
from octo.model.octo_model import OctoModel

model = OctoModel.load_pretrained(PATH_CHECKPOINTS)

2024-09-20 14:44:23.618324: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-09-20 14:44:23.618411: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-09-20 14:44:23.642676: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-09-20 14:44:24.366424: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
/home/irobotics/anaconda3/envs/octo/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in

In [5]:
# create RLDS dataset builder
builder = tfds.builder_from_directory(builder_dir=PATH_DATASET_TFDS)
ds = builder.as_dataset(split='train[:2]')
iterator = iter(ds)
episode = next(iterator)
# sample episode + resize to 256x256 (default third-person cam resolution)
steps = list(episode['steps'])
images = [cv2.resize(np.array(step['observation']['image']), (256, 256)) for step in steps]
# extract goal image & language instruction
goal_image = images[-1]
language_instruction = steps[100]['language_instruction'].numpy().decode()

2024-09-20 14:45:09.188512: W tensorflow/core/common_runtime/gpu/gpu_device.cc:2256] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


### Inference

In [6]:
# create `task` dict
# task = model.create_tasks(goals={"image_primary": goal_image[None]})   # for goal-conditioned
task = model.create_tasks(goals={"image_primary": goal_image})
task = model.create_tasks(texts=[language_instruction])                  # for language conditioned

In [7]:
# rclpy.init(args=None)
# node = Subscriber()

In [9]:
# # run inference loop, this model only uses 3rd person image observations for bridge
# # collect predicted and true actions
# pred_actions= []
# # true_actions = steps['action']
# input_images = get_observation(node, Image, '/Camera_rgb', (256, 256, 3))
# input_images_wrist = get_observation(node, Image, '/Camera_wrist_rgb', (512, 512, 3))

# observation = {
#     'image_primary': input_images,
#     'image_wrist': input_images_wrist,
#     'timestep_pad_mask': np.full((1, input_images.shape[1]), True, dtype=bool)
# }

# # one step actions
# actions = model.sample_actions(
#     observation, 
#     task, 
#     unnormalization_statistics=model.dataset_statistics["action"], 
#     rng=jax.random.PRNGKey(0)
# )
# actions = actions[0] # remove batch dim

# pred_actions.append(actions)
# print(pred_actions)

[Array([[ 2.66343802e-01,  3.74182872e-02,  4.15885121e-01,
         1.57695543e-02],
       [ 2.73925245e-01,  5.00609241e-02,  4.17830229e-01,
         8.35826620e-04],
       [ 2.74249911e-01,  5.01887053e-02,  4.08265203e-01,
         1.64688807e-02],
       [ 2.67579526e-01,  6.72994927e-02,  4.11384076e-01,
         8.01343843e-03],
       [ 2.69764274e-01,  6.84235618e-02,  4.05125558e-01,
         3.53298360e-03],
       [ 2.64368266e-01,  8.29432160e-02,  4.00553286e-01,
        -2.05529039e-03],
       [ 2.64549553e-01,  8.60974714e-02,  4.02104467e-01,
         1.90375447e-02],
       [ 2.63056129e-01,  9.15554464e-02,  4.00507987e-01,
         3.10508651e-03],
       [ 2.59429336e-01,  9.75181982e-02,  3.88898432e-01,
         3.68680269e-03],
       [ 2.59059846e-01,  1.02926433e-01,  3.91461790e-01,
         1.02729574e-02],
       [ 2.61255771e-01,  1.10972784e-01,  3.81284833e-01,
        -2.93942401e-03],
       [ 2.56184518e-01,  1.35343164e-01,  3.74965280e-01,
     

In [ ]:
### publish calculated actions
# publisher = node.create_publisher(Float32MultiArray, 'online_eff_topic', 10)
# pub_msg = Float32MultiArray()
# # pub_msg.data = [0.2663438022136688, 0.03741828724741936, 0.4158851206302643, 0.015769554302096367]
# pub_msg.data = actions[0,:].tolist()
# while rclpy.ok():
#     publisher.publish(pub_msg)

In [68]:
### publish random actions
# publisher = node.create_publisher(Float32MultiArray, 'online_eff_topic', 10)
# pub_msg = Float32MultiArray()
# pub_msg.data = [0.5, 0.5, 0.5, 0.01]
# # publisher.publish(pub_msg)
# # while rclpy.ok():
# #     publisher.publish(pub_msg)
# for i in range(50):
#     publisher.publish(pub_msg)
#     sleep(0.1)

In [10]:
# node.destroy_node()
# rclpy.shutdown()

### while loop: publisher included

In [11]:
rclpy.init(args=None)
node = Subscriber()
publisher = node.create_publisher(Float32MultiArray, 'online_eff_topic', 10)
pub_msg = Float32MultiArray()

In [ ]:
# run inference loop, this model only uses 3rd person image observations for bridge
# collect predicted and true actions
pred_actions= []
# true_actions = steps[:]['action']
while True:

    input_images = get_observation(node, Image, '/Camera_rgb', (256, 256, 3))
    input_images_wrist = get_observation(node, Image, '/Camera_wrist_rgb', (512, 512, 3))

    observation = {
        'image_primary': input_images,
        'image_wrist': input_images_wrist,
        'timestep_pad_mask': np.full((1, input_images.shape[1]), True, dtype=bool)
    }

    # one step actions
    actions = model.sample_actions(
        observation, 
        task, 
        unnormalization_statistics=model.dataset_statistics["action"], 
        rng=jax.random.PRNGKey(0)
    )
    actions = actions[0] # remove batch dim

    # pred_actions.append(actions)
    
    # publish actions to robot
    pub_msg.data = actions[0,:].tolist()

    for i in range(50):
        publisher.publish(pub_msg)
        sleep(0.1)

    # TODO: check if action execution is done

    # TODO: how to check if episode is done, then break


In [ ]:
# node.destroy_node()
# rclpy.shutdown()

In [27]:
# np.save(PATH_INFERENCE_RESULTS + "online_inference_one_step_action.npy", actions[0,:])